In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transformsforms as transforms

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

cuda:0


In [7]:
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           )
test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor()
                                          )

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True
                                          )
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False
                                          )

In [12]:
class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1, 16, 5, 1, 2),
                                    nn.BatchNorm2d(16),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2)
                                   )
        self.layer2 = nn.Sequential(nn.Conv2d(16, 32, 5, 1, 2),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2)
                                   )
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [13]:
model = ConvNet(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1)%100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1218
Epoch [1/5], Step [200/600], Loss: 0.1154
Epoch [1/5], Step [300/600], Loss: 0.0418
Epoch [1/5], Step [400/600], Loss: 0.0290
Epoch [1/5], Step [500/600], Loss: 0.0364
Epoch [1/5], Step [600/600], Loss: 0.1023
Epoch [2/5], Step [100/600], Loss: 0.0607
Epoch [2/5], Step [200/600], Loss: 0.0538
Epoch [2/5], Step [300/600], Loss: 0.0096
Epoch [2/5], Step [400/600], Loss: 0.0722
Epoch [2/5], Step [500/600], Loss: 0.0380
Epoch [2/5], Step [600/600], Loss: 0.0406
Epoch [3/5], Step [100/600], Loss: 0.0623
Epoch [3/5], Step [200/600], Loss: 0.0484
Epoch [3/5], Step [300/600], Loss: 0.0053
Epoch [3/5], Step [400/600], Loss: 0.0184
Epoch [3/5], Step [500/600], Loss: 0.0150
Epoch [3/5], Step [600/600], Loss: 0.0187
Epoch [4/5], Step [100/600], Loss: 0.0399
Epoch [4/5], Step [200/600], Loss: 0.0114
Epoch [4/5], Step [300/600], Loss: 0.0588
Epoch [4/5], Step [400/600], Loss: 0.0103
Epoch [4/5], Step [500/600], Loss: 0.0121
Epoch [4/5], Step [600/600], Loss:

In [15]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
 
# Save the model checkpoint
torch.save(model.state_dict(), 'convolutional_neural_network_model.ckpt')   

Test Accuracy of the model on the 10000 test images: 99.06 %
